# Collections, bedingte Anweisungen, Schleifen, List Comprehensions
## Verwalte den Python-Kurs
Die Datei Lektionen_1_bis_3_Studierende.txt enthält Studierenden-Daten nach folgendem Schema: \<Name\>; \<Studiengang\>; \<Semester\>; \<Hintergrund\>. Der Name ist eindeutig. Der Hintergrund-Eintrag kann verschiedene kommaseparierte Informationen enthalten. Pro Zeile sind die Daten einer Person erfasst. Du kannst die Datei in einem Texteditor öffnen, um dir das Format und die Inhalte anzuschauen.
## Aufgabe A
Lies die Daten aus der Datei ein und speichere sie strukturiert (unter Verwendung geeigneter Collections, z.B. Dictionary und Liste) in einer Variablen `python_course`. 

Tipps: 
* Recherchiere [hier](https://docs.python.org/3/tutorial/inputoutput.html#methods-of-file-objects), wie du Dateien zeilenweise einlesen kannst. 
* Erinnere dich an die String-Funktion, mit der du einen String "zerschneiden" kannst.

In [ ]:
python_course = [] #gesamter Kurs als Liste
keys = ("Name", "Studiengang", "Semester", "Hintergrund") #Schlüssel als Tupel
filename = "Lektionen_1_bis_3_Studierende.txt"

with open(filename, "r", encoding="utf-8") as file:
    for line in file:
        items = line.strip().split("; ") #strip entfernt das finale Newline
        #jede Person als Dictionary
        python_course.append({keys[0]:items[0], keys[1]:items[1], keys[2]:items[2], keys[3]:items[3]})
print(python_course)

## Aufgabe B
* Gib die Teilnehmenden strukturiert (eine Person pro Zeile) aus
* Berechne das durchschnittliche Semester aller Teilnehmenden und gib dieses aus

In [ ]:
sum_semester = 0
for person in python_course:
    line = ""
    for i in range(4):
        line += keys[i]+": "+person[keys[i]]+", "
    print(line)
    sum_semester += int(person[keys[2]])
print("\nDas durchschnittliche Semester aller Teilnehmenden ist:", sum_semester/len(python_course))

## Aufgabe C
* Erstelle eine Liste der Namen aller Erstsemester unter Verwendung einer List Comprehension und gib diese aus

In [ ]:
first_semester = [person[keys[0]] for person in python_course if int(person[keys[2]]) == 1]
print("Die Namen der Studierenden im ersten Semester lauten:", first_semester)

## Aufgabe D (Weiterführende Hausaufgabe)
* Fordere die Eingabe eines Namens aus dem Python-Kurs (mittels input). 
* Schlage für diese Person andere Kursteilnehmende vor, mit denen sie sich vernetzen kann. Die Vorschläge sollen aufgrund von Gemeinsamkeiten generiert werden.

**Mögliche Vorgehensweise:**
* Mögliche Gemeinsamkeiten könnten der Studiengang oder Hintergrundinformationen sein.
* Um eine effiziente Implementierung zu erhalten (so dass nicht alle Relationen bei jeder Anfrage neu berechnet werden müssen), erstelle zunächst zwei Indizes als Dictionaries (dies muss nur einmal durchgeführt werden): 
    1) Schlüssel: Name, Wert: Liste möglicher Gemeinsamkeiten, z.B. in der Variablen `communalities_by_name` vom Typ [defaultdict](https://docs.python.org/3/library/collections.html#collections.defaultdict) 
    2) Schlüssel: mögliche Gemeinsamkeit, Wert: Liste der Namen, die diese aufweisen, z.B. in der Variablen `names_by_communality` vom Typ [defaultdict](https://docs.python.org/3/library/collections.html#collections.defaultdict) 

In [ ]:
# Vorbereitung: Einmalige Erstellung der Indizes
# defaultdict entspricht einem Dictionary, mit dem Unterschied, dass es einen definierbaren Default-Wert liefert, 
# falls ein angefragter Schlüssel nicht vorhanden ist
from collections import defaultdict 

# Erstelle zunächst die Indizes: 
# 1) Generiere für jede Person eine Liste möglicher Gemeinsamkeiten aus den Angaben für diese Person und speichere
#    dies in einem defaultdict
communalities_by_name = defaultdict(list) # liefert eine leere Liste, falls angefragter Schlüssel nicht vorhanden
for person in python_course:
    # Sammle alle möglichen Gemeinsamkeiten zu der aktuellen Person
    # Der Name der Person ist der Schlüssel, der Wert die Liste der möglichen Gemeinsamkeiten
    # casefold, um robust ggü. verschiedenen Schreibweisen zu sein
    communalities_by_name[person[keys[0]]].append(person[keys[1]].casefold()) # Studiengang
    background = person[keys[3]].casefold(); # Hintergrund, kommaseparierte Informationen
    backgrounds = background.split(",")
    for item in backgrounds:
        communalities_by_name[person[keys[0]]].append(item.strip()) # jede Information aus dem Hintergrund
    #print(communalities_by_name[person[keys[0]]])

# 2) Generiere für jede mögliche Gemeinsamkeit eine Liste der Personen, die diese aufweisen
names_by_communality = defaultdict(list)
for name in communalities_by_name:
    for communality in communalities_by_name[name]:
        names_by_communality[communality].append(name)
    #print(communality, names_by_communality[communality])
    
print(communalities_by_name)
print(names_by_communality)

**Mögliche Vorgehensweise (cont'd):**
* Fordere die Eingabe eines Namens und prüfe, ob dieser Name bekannt ist
* Wenn ja, verwende die aufgebauten Indizes, um Namen zu finden, mit denen die eingegebene Person Gemeinsamkeiten hat, und speichere diese jeweils ab. Dafür kann ebenfalls ein defaultdict verwendet werden.
* Gib alle gefundenen Namen sowie die Gemeinsamkeiten aus

In [ ]:
# Anfrage und Ausgabe
# Dieser Block kann mehrfach mit unterschiedlichen Namen ausgeführt werden, 
# wobei die Indizes nur einmal berechnet werden
query_name = ""
while(True):
    query_name = input("Für wen sollen Vernetzungsmöglichkeiten gesucht werden? Bitte geben Sie einen Namen ein:")  
    if query_name in communalities_by_name:
        break;
    else:
        print("Name", query_name, "nicht bekannt")
# Hier ist klar, dass der angefragte Name bekannt ist

# Das folgende defaultdict wird nur für die angefragte Person aufgebaut
# Schlüssel: Name einer anderen Person, Wert: Liste der Gemeinsamkeiten zwischen dieser und der angefragten Person
communalities_for_query_name = defaultdict(list)

# Für jede mögliche Gemeinsamkeit der angefragten Person
for communality in communalities_by_name[query_name]:
    # Für jeden Namen, der diese Gemeinsamkeit auch aufweist
    for name in names_by_communality[communality]:
        if name != query_name: # sofern es nicht der Name der angefragten Person selbst ist
            #print(query_name, name, communality)
            communalities_for_query_name[name].append(communality) # speichere dies

if communalities_for_query_name: # falls nicht leer
    print("Mit folgenden Personen hat",query_name,"Gemeinsamkeiten:")
    for name in communalities_for_query_name:
        print("\tName:", name, ", Gemeinsamkeit(en):", communalities_for_query_name[name])